# Actuarial Analysis Methods with Chainladder AI Agent

This notebook demonstrates how to use the Chainladder AI Agent to perform various actuarial analyses. We'll explore development methods, IBNR calculations, and more.

In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import chainladder as cl

# Add the project root to the path so we can import our package
sys.path.append('..')

# Import the supervisor agent creator
from chainladder_agent.agents.supervisor import create_chainladder_supervisor

# Get API key
api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    print("Warning: No OpenAI API key found. Please set your API key.")
else:
    print("API key is set.")

# Force reload the analysis_tools module to get the latest implementation
import importlib
from chainladder_agent.tools import analysis_tools
importlib.reload(analysis_tools)

# Create the supervisor agent
supervisor = create_chainladder_supervisor(api_key=api_key)

API key is set.


## Helper Function for Agent Interaction

In [5]:

def ask_agent(query, triangle_name="", thread_id=None):
    """Helper function to interact with the Chainladder AI Agent.
    
    Parameters:
        query (str): The question or instruction for the agent
        triangle_name (str): Optional name of the triangle dataset to use
        thread_id (str): Optional thread ID for maintaining conversation state
        
    Returns:
        str: The agent's response
    """
    # Prepare the input
    input_data = {
        "messages": [{"role": "user", "content": query}],
        "selected_triangle": triangle_name
    }
    
    # Prepare config with thread_id for the checkpointer
    config = {}
    if thread_id:
        config["configurable"] = {"thread_id": thread_id}
    else:
        # Create a simple thread ID based on the triangle name and query
        simple_id = f"notebook_{triangle_name}_{hash(query) % 10000}"
        config["configurable"] = {"thread_id": simple_id}
    
    # Run the query with thread_id config
    result = supervisor.invoke(input_data, config=config)

    # Handle the result based on its structure
    if isinstance(result, dict) and "messages" in result:
        # Extract the AI messages from the response
        ai_messages = []
        for message in result.get("messages", []):
            if hasattr(message, 'type') and message.type == 'ai':
                if message.content and len(message.content) > 20:
                    ai_messages.append((len(message.content), message.content))
        
        # Sort by content length and get the longest (likely most informative) response
        if ai_messages:
            ai_messages.sort(reverse=True)
            return ai_messages[0][1]
        else:
            return "No response received."
    else:
        # If it's already a string or other format, return as is
        return result


## 1. Understanding Development Methods

Development methods are used to estimate how losses develop over time. Let's start by asking the agent to explain development methods:

In [3]:
development_explanation = ask_agent("Explain what development methods are in actuarial science and how they're used.")
print(development_explanation)

{'messages': [HumanMessage(content="Explain what development methods are in actuarial science and how they're used.", additional_kwargs={}, response_metadata={}, id='2689ff1f-a868-4af1-90bb-67b8b0fc85ca'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nppniEXQMO18KLLvf2BN06qc', 'function': {'arguments': '{}', 'name': 'transfer_to_explanation_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 395, 'total_tokens': 409, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_a1102cf978', 'id': 'chatcmpl-BTqkazcFTBxiTkNmBL7PdetbenwvX', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run--6a1d15fd-3acb-42ee-835b-786cd037cc4a-0', tool_c

## 2. Applying the Chain Ladder Method

Now, let's apply the Chain Ladder method to the RAA triangle dataset:

In [6]:
chain_ladder_analysis = ask_agent("What is the Chain Ladder reserve estimate for the raa triangle?  Include the LDFs, latest diagonals, ultimate losses by origin year. No visuals.", "raa", 1)
print(chain_ladder_analysis)

Here are the Chain Ladder reserve estimates for the RAA triangle, as requested:

Loss Development Factors (LDFs):
- 12-24: 2.9994
- 24-36: 1.6235
- 36-48: 1.2709
- 48-60: 1.1717
- 60-72: 1.1134
- 72-84: 1.0419
- 84-96: 1.0333
- 96-108: 1.0169
- 108-120: 1.0092

Latest Diagonal (Paid to date by origin year):
- 1981: 18,834.0
- 1982: 16,704.0
- 1983: 23,466.0
- 1984: 27,067.0
- 1985: 26,180.0
- 1986: 15,852.0
- 1987: 12,314.0
- 1988: 13,112.0
- 1989: 5,395.0
- 1990: 2,063.0

Ultimate Losses by Origin Year:
- 1981: 18,834.0
- 1982: 16,857.95
- 1983: 24,083.37
- 1984: 28,703.14
- 1985: 28,926.74
- 1986: 19,501.10
- 1987: 17,749.30
- 1988: 24,019.19
- 1989: 16,044.98
- 1990: 18,402.44

Total Ultimate Losses: 213,122.23  
Total IBNR: 52,135.23

If you need further explanation or a breakdown of the calculations, let me know!


## 3. Understanding Loss Development Factors (LDFs)

Let's get a more detailed explanation of Loss Development Factors (LDFs):

In [ ]:
ldf_explanation = ask_agent("What are Loss Development Factors (LDFs)? How are they calculated and interpreted?")
print(ldf_explanation)

## 4. Different Averaging Methods for Development Factors

There are several methods for calculating development factors. Let's ask the agent about them:

In [ ]:
averaging_methods = ask_agent("What different averaging methods are available for calculating development factors in the chainladder package? Compare volume-weighted, simple, and regression methods.")
print(averaging_methods)

## 5. Calculating IBNR with the Chain Ladder Method

Let's calculate the Incurred But Not Reported (IBNR) reserves using the Chain Ladder method:

In [5]:
chain_ladder_ibnr = ask_agent("Calculate IBNR using the Chain Ladder method on the RAA triangle. Show the ultimate losses and IBNR by origin year.", "raa")
print(chain_ladder_ibnr)

The RAA triangle is a classic example used in actuarial analysis. Here’s a summary of its structure and validation:

- Shape: 10 origin years × 10 development periods (yearly grain)
- Origin periods: 1981 to 1990
- Development periods: 12, 24, ..., 120 months (1 to 10 years)
- Columns: "values" (typically paid or incurred losses)
- Format: Cumulative
- The triangle is valid and suitable for Chain Ladder analysis.

Next, I will calculate the ultimate losses and IBNR (Incurred But Not Reported) by origin year using the Chain Ladder method.
To calculate IBNR and ultimate losses using the Chain Ladder method, I will:

1. Extract the latest diagonal (most recent cumulative value for each origin year).
2. Calculate development factors for each development period.
3. Project each origin year to ultimate using these factors.
4. Compute IBNR as the difference between the projected ultimate and the latest diagonal.

Let me proceed with these calculations for the RAA triangle.


## 6. Mack Chain Ladder Method

The Mack Chain Ladder method extends the traditional Chain Ladder method by adding uncertainty estimates:

In [ ]:
mack_analysis = ask_agent("Apply the Mack Chain Ladder method to the RAA triangle. Show the uncertainty estimates and explain how they're calculated.", "raa")
print(mack_analysis)

## 7. Bornhuetter-Ferguson Method

The Bornhuetter-Ferguson method is another important actuarial method that blends Chain Ladder with expected loss ratios:

In [ ]:
bf_explanation = ask_agent("What is the Bornhuetter-Ferguson method? How does it differ from the Chain Ladder method?")
print(bf_explanation)

In [ ]:
bf_analysis = ask_agent("Apply the Bornhuetter-Ferguson method to the GenIns triangle. Compare the results with the Chain Ladder method.", "genins")
print(bf_analysis)

## 8. Benktander Method

The Benktander method is sometimes called a credibility-weighted average of Chain Ladder and Bornhuetter-Ferguson:

In [ ]:
benktander_explanation = ask_agent("What is the Benktander method and how is it related to Chain Ladder and Bornhuetter-Ferguson methods?")
print(benktander_explanation)

In [ ]:
benktander_analysis = ask_agent("Apply the Benktander method to the GenIns triangle and explain the results.", "genins")
print(benktander_analysis)

## 9. Cape Cod Method

Let's also explore the Cape Cod method:

In [ ]:
cape_cod_explanation = ask_agent("What is the Cape Cod method in actuarial science? How does it work?")
print(cape_cod_explanation)

In [ ]:
cape_cod_analysis = ask_agent("Apply the Cape Cod method to the RAA triangle and explain the results.", "raa")
print(cape_cod_analysis)

## 10. Bootstrap Analysis

Bootstrap analysis provides a way to estimate prediction uncertainty:

In [ ]:
bootstrap_explanation = ask_agent("Explain what bootstrap analysis is in the context of actuarial reserving. How does it help with uncertainty estimation?")
print(bootstrap_explanation)

In [ ]:
bootstrap_analysis = ask_agent("Perform a bootstrap analysis on the RAA triangle with 1000 simulations. Explain the percentiles of the results.", "raa")
print(bootstrap_analysis)

## 11. Comparison of Methods

Let's compare different methods side by side:

In [ ]:
methods_comparison = ask_agent("Compare the Chain Ladder, Bornhuetter-Ferguson, Benktander, and Cape Cod methods on the RAA triangle. Show the total IBNR from each method and explain when each method is most appropriate.", "raa")
print(methods_comparison)

## 12. Manual Implementation of Chain Ladder

Let's also implement the Chain Ladder method manually to understand the calculations better:

In [ ]:
# Load the RAA triangle
raa = cl.load_sample.raa()
print("RAA Triangle:")
display(raa.to_frame())

# Apply Development
dev = cl.Development().fit(raa)
print("\nLoss Development Factors (LDFs):")
display(dev.ldf_.to_frame())

# Apply Chain Ladder
cl_model = cl.Chainladder().fit(dev)
print("\nUltimate Losses:")
display(cl_model.ultimate_.to_frame())

print("\nIBNR:")
display(cl_model.ibnr_.to_frame())

print(f"\nTotal IBNR: {cl_model.ibnr_.sum().sum()}")

## 13. Tail Factors

When triangles don't fully develop within the available development periods, we need to apply tail factors:

In [ ]:
tail_explanation = ask_agent("What are tail factors in actuarial reserving? Why are they important and how are they calculated?")
print(tail_explanation)

In [ ]:
tail_application = ask_agent("Apply a constant tail factor of 1.05 to the RAA triangle after Chain Ladder development. How does this affect the IBNR?", "raa")
print(tail_application)

## Summary

In this notebook, we've explored how to use the Chainladder AI Agent to perform various actuarial analyses:

1. Development Methods and Loss Development Factors (LDFs)
2. Chain Ladder Method
3. Mack Chain Ladder with uncertainty estimates
4. Bornhuetter-Ferguson Method
5. Benktander Method
6. Cape Cod Method
7. Bootstrap Analysis for uncertainty estimation
8. Comparison of different methods
9. Tail factor application

The agent provides both explanations of the methods and practical applications to triangle data, helping actuaries understand the underlying principles and interpret the results.

In the next notebook, we'll explore how to use the Chainladder AI Agent for visualizing triangle data and analysis results.